In [338]:
import pandas as pd
import dill as pickle
import networkx as nx
import itertools
from collections import Counter
import heapq
import time

In [259]:
# read in 3 files 
scores = pd.read_csv('sample_scores.csv')
friends = pd.read_csv('friends.csv')
foes = pd.read_csv('foes.csv')

In [12]:
friends.head()

,person1,person2
0,441e08b1_a0431c52,404021bf_898326ec
1,3f4929ef_956613b7,d6031447_a684b614
2,1f75eb7f_e1d4405b,5ccf7530_5343d32b
3,9b6edd00_179def5,e7d1222f_49ff1c63
4,130181c4_27cb3b23,536c3152_db65659b


In [13]:
foes.head()

,person1,person2
0,1a78a273_69afff35,cbbb389d_6c185226
1,b49793f1_89131c25,528b6768_4cfc6dfc
2,dae7f2ef_a0eb0f08,f855223d_12422aaf
3,653464a_6d49face,7ac0271c_3677a2d6
4,ae33fd68_8c646d91,b66507c7_44b175d3


In [19]:
scores.head()

,person id,score
0,1122b7d5_ed317db7,-2
1,b0177400_be9028c7,-2
2,77ac4e2a_77109d8c,-1
3,49ca4e7d_2eb499f,2
4,9e4a7320_9ebed277,-2


In [22]:
list(scores)

['person id', 'score']

In [15]:
len(scores)

1927

In [16]:
len(friends)

474835

In [17]:
len(foes)

135644

## Part (a)

### Merging foes and scores (left merge on person id)

In [23]:
f1 = foes.merge(scores, how = 'left', left_on = 'person1', right_on = 'person id')

In [30]:
f1.head()

,person1,person2,person id,score
0,1a78a273_69afff35,cbbb389d_6c185226,NaN,NaN
1,b49793f1_89131c25,528b6768_4cfc6dfc,NaN,NaN
2,dae7f2ef_a0eb0f08,f855223d_12422aaf,NaN,NaN
3,653464a_6d49face,7ac0271c_3677a2d6,NaN,NaN
4,ae33fd68_8c646d91,b66507c7_44b175d3,NaN,NaN


In [34]:
f1 = f1.drop('person id', axis = 1)
f1 = f1.rename(columns = {"score": "score1"})

In [35]:
f1.head()

,person1,person2,score1
0,1a78a273_69afff35,cbbb389d_6c185226,NaN
1,b49793f1_89131c25,528b6768_4cfc6dfc,NaN
2,dae7f2ef_a0eb0f08,f855223d_12422aaf,NaN
3,653464a_6d49face,7ac0271c_3677a2d6,NaN
4,ae33fd68_8c646d91,b66507c7_44b175d3,NaN


In [36]:
len(f1)

135644

In [37]:
f2 = f1.merge(scores, how = 'left', left_on = 'person2', right_on = 'person id')

In [38]:
f2.head()

,person1,person2,score1,person id,score
0,1a78a273_69afff35,cbbb389d_6c185226,NaN,NaN,NaN
1,b49793f1_89131c25,528b6768_4cfc6dfc,NaN,NaN,NaN
2,dae7f2ef_a0eb0f08,f855223d_12422aaf,NaN,NaN,NaN
3,653464a_6d49face,7ac0271c_3677a2d6,NaN,NaN,NaN
4,ae33fd68_8c646d91,b66507c7_44b175d3,NaN,NaN,NaN


In [39]:
f2 = f2.drop('person id', axis = 1)
f2 = f2.rename(columns = {"score": "score2"})

In [41]:
f2.head()

,person1,person2,score1,score2
0,1a78a273_69afff35,cbbb389d_6c185226,NaN,NaN
1,b49793f1_89131c25,528b6768_4cfc6dfc,NaN,NaN
2,dae7f2ef_a0eb0f08,f855223d_12422aaf,NaN,NaN
3,653464a_6d49face,7ac0271c_3677a2d6,NaN,NaN
4,ae33fd68_8c646d91,b66507c7_44b175d3,NaN,NaN


In [42]:
len(f2)

135644

In [51]:
f2.describe()

,score1,score2
count,135644.000000,135644.000000
mean,0.006768,-0.030175
std,0.919822,0.867689
min,-4.000000,-4.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,5.000000,5.000000


In [44]:
foes_new = f2

In [48]:
## fill NA with 0
foes_new['score1'].fillna(0, inplace=True)

In [50]:
foes_new['score2'].fillna(0, inplace=True)

In [63]:
foes_q = foes_new[(foes_new['score1'] > 0) & (foes_new['score2'] > 0)]

In [65]:
len(foes_q)

259

In [70]:
len(foes_new)

135644

In [69]:
foes_q[:20]     # verified based on original tables 

,person1,person2,score1,score2
0,2ce1f07_cdd4f17e,258e3ae3_afaf2804,2.0,2.0
1,718ff97d_875e9587,12318fc_80f41db,2.0,2.0
2,bb891fa8_b2f18316,326fced_b2cbc85,2.0,2.0
3,11d46322_2c18b95d,2948ed6e_eb231cf7,2.0,2.0
4,612e3f7a_3ce2519d,34ddbc2f_91d54040,2.0,2.0
5,2859f13a_7f6124d7,8004eba5_3b3e18e8,1.0,2.0
6,479ee8d3_37fac876,b1d619fd_7357f9ab,2.0,2.0
7,aaf0dc33_5377b4f9,5a84de_ff37c82d,1.0,1.0
8,1801d773_ad28c52f,e1ffddcf_e29638f4,1.0,2.0
9,3660dca9_76de4216,f94fdac2_e2370bcb,3.0,2.0


In [68]:
foes_q.reset_index(drop=True, inplace = True)

In [71]:
### pickle foes_new and foes_q
with open('foes_new.pkl', 'wb') as file:
    pickle.dump(foes_new, file)


In [74]:
with open('foes_q.pkl', 'wb') as file:
    pickle.dump(foes_q, file)

In [73]:
#foes_new.head()

### Merging friends and scores (left merge on person id)

In [3]:
f1 = friends.merge(scores, how = 'left', left_on = 'person1', right_on = 'person id')

In [7]:
f1.head()

,person1,person2,score1
0,441e08b1_a0431c52,404021bf_898326ec,NaN
1,3f4929ef_956613b7,d6031447_a684b614,NaN
2,1f75eb7f_e1d4405b,5ccf7530_5343d32b,NaN
3,9b6edd00_179def5,e7d1222f_49ff1c63,NaN
4,130181c4_27cb3b23,536c3152_db65659b,NaN


In [6]:
f1 = f1.drop('person id', axis = 1)
f1 = f1.rename(columns = {"score": "score1"})

In [8]:
len(f1)

474835

In [9]:
f2 = f1.merge(scores, how = 'left', left_on = 'person2', right_on = 'person id')

In [10]:
f2 = f2.drop('person id', axis = 1)
f2 = f2.rename(columns = {"score": "score2"})

In [11]:
f2.head()

,person1,person2,score1,score2
0,441e08b1_a0431c52,404021bf_898326ec,NaN,NaN
1,3f4929ef_956613b7,d6031447_a684b614,NaN,NaN
2,1f75eb7f_e1d4405b,5ccf7530_5343d32b,NaN,NaN
3,9b6edd00_179def5,e7d1222f_49ff1c63,NaN,NaN
4,130181c4_27cb3b23,536c3152_db65659b,NaN,NaN


In [12]:
f2.describe()

,score1,score2
count,56788.000000,49027.000000
mean,-0.805786,-0.652171
std,1.916858,1.985760
min,-4.000000,-4.000000
25%,-2.000000,-2.000000
50%,-2.000000,-2.000000
75%,1.000000,2.000000
max,5.000000,5.000000


In [13]:
friends_new = f2

In [18]:
f2.head()

,person1,person2,score1,score2
0,441e08b1_a0431c52,404021bf_898326ec,NaN,NaN
1,3f4929ef_956613b7,d6031447_a684b614,NaN,NaN
2,1f75eb7f_e1d4405b,5ccf7530_5343d32b,NaN,NaN
3,9b6edd00_179def5,e7d1222f_49ff1c63,NaN,NaN
4,130181c4_27cb3b23,536c3152_db65659b,NaN,NaN


In [19]:
friends_new['score1'].fillna(0, inplace=True)
friends_new['score2'].fillna(0, inplace=True)

In [20]:
friends_new.head()

,person1,person2,score1,score2
0,441e08b1_a0431c52,404021bf_898326ec,0.0,0.0
1,3f4929ef_956613b7,d6031447_a684b614,0.0,0.0
2,1f75eb7f_e1d4405b,5ccf7530_5343d32b,0.0,0.0
3,9b6edd00_179def5,e7d1222f_49ff1c63,0.0,0.0
4,130181c4_27cb3b23,536c3152_db65659b,0.0,0.0


In [26]:
friends_new[(friends_new['score1'] != 0) & (friends_new['score2'] != 0)].head()    # verified!

,person1,person2,score1,score2
67,694d36b5_a4804f20,23bf40f9_39fcc80c,-4.0,-4.0
162,b6156075_219901ae,6c4e4047_afe4cf71,-2.0,-2.0
228,ba335400_fb20ba06,1632bec2_52d70733,-3.0,-3.0
266,7077ed63_de96f676,352e8c74_919a8d,3.0,-2.0
387,4a2bd5e2_8deb71c8,4b5451ee_2ef13224,-1.0,-2.0


In [24]:
#friends_new.loc[friends_new['score1']!= 0].head()

In [28]:
len(friends_new)

474835

In [29]:
with open('friends_new.pkl', 'wb') as file:
    pickle.dump(friends_new, file)

### removing false records in foe table

In [40]:
with open('foes_new.pkl', 'rb') as file:
    foes = pickle.load(file)

In [30]:
with open('foes_q.pkl', 'rb') as file:
    foes_q = pickle.load(file)

In [57]:
len(foes_q)

259

In [58]:
len(foes)

135644

In [66]:
foes_q = foes[(foes['score1'] > 0) & (foes['score2'] > 0)]

In [67]:
foes_q.head()

,person1,person2,score1,score2
329,2ce1f07_cdd4f17e,258e3ae3_afaf2804,2.0,2.0
432,718ff97d_875e9587,12318fc_80f41db,2.0,2.0
1123,bb891fa8_b2f18316,326fced_b2cbc85,2.0,2.0
1215,11d46322_2c18b95d,2948ed6e_eb231cf7,2.0,2.0
1298,612e3f7a_3ce2519d,34ddbc2f_91d54040,2.0,2.0


In [68]:
len(foes_q)

259

In [69]:
# true foes
foes_true = foes[~foes.index.isin(foes_q.index)]    

In [70]:
foes_true.head()

,person1,person2,score1,score2
0,1a78a273_69afff35,cbbb389d_6c185226,0.0,0.0
1,b49793f1_89131c25,528b6768_4cfc6dfc,0.0,0.0
2,dae7f2ef_a0eb0f08,f855223d_12422aaf,0.0,0.0
3,653464a_6d49face,7ac0271c_3677a2d6,0.0,0.0
4,ae33fd68_8c646d91,b66507c7_44b175d3,0.0,0.0


In [71]:
len(foes_true)    # make sense: 135644 - 259 = 135385

135385

In [72]:
foes_true.reset_index(drop=True, inplace = True)

In [74]:
#foes_true

In [75]:
# pickle true foes
with open('foes_true.pkl', 'wb') as file:
    pickle.dump(foes_true, file)

### subset of friends and foes (at least one score is non-zero in each row)####

In [9]:
with open('foes_true.pkl', 'rb') as file:
    foes = pickle.load(file)

In [6]:
with open('friends_new.pkl', 'rb') as file:
    friends = pickle.load(file)

In [10]:
len(foes)

135385

In [13]:
friends_s = friends[(friends['score1'] != 0) | (friends['score2'] != 0) ]

In [15]:
len(friends_s)

97196

In [18]:
friends_s.reset_index(drop=True, inplace = True)

In [19]:
friends_s.head()

,person1,person2,score1,score2
0,6cb34607_72386def,8260e1c3_f518feae,0.0,-2.0
1,556ba434_c0ca2371,c38d1b99_3d61be1a,-2.0,0.0
2,9a04f2c1_2f5068c8,ac7d8101_726035f,0.0,-2.0
3,1c9221db_22904429,c42f7784_19bb34eb,0.0,-1.0
4,36f4ed0a_ae3f03e9,beee874_b0e1c2c9,-2.0,0.0


In [11]:
foes_s = foes[(foes['score1'] != 0) | (foes['score2'] != 0)]

In [12]:
foes_s.reset_index(drop=True, inplace = True)

In [13]:
len(foes_s)

45488

In [31]:
foes_s.head()

,person1,person2,score1,score2
0,8b28e2cf_2b895a86,1fca0d07_adc649b8,-1.0,0.0
1,97688328_79a98884,3660dca9_76de4216,0.0,3.0
2,916c397c_eb9d1676,e1ffddcf_e29638f4,0.0,2.0
3,ba745a27_ecf057dc,d34748d2_55b76015,0.0,-2.0
4,76effdcf_a75d185e,2a663754_d6c9ffa3,-1.0,0.0


In [14]:
### pickle foes_s and friends_s
with open('foes_s.pkl', 'wb') as file:
    pickle.dump(foes_s, file)

In [26]:
with open('friends_s.pkl', 'wb') as file:
    pickle.dump(friends_s, file)

#### build a dictionay with elements in small datasets#### 

In [28]:
with open('friends_s.pkl', 'rb') as file:
    friends_s = pickle.load(file)

In [29]:
with open('foes_s.pkl', 'rb') as file:
    foes_s = pickle.load(file)

In [30]:
# create a dictionary:
id_score_s = {}

In [31]:
for index, row in foes_s.iterrows():
    if row['person1'] not in id_score_s: 
        id_score_s[row['person1']] = row['score1']
    if row['person2'] not in id_score_s: 
        id_score_s[row['person2']] = row['score2']  

In [32]:
len(id_score_s.keys())   # make sense

12886

In [33]:
id_score_s['8b28e2cf_2b895a86']
id_score_s['d34748d2_55b76015']

-2.0

In [33]:
for index, row in friends_s.iterrows():
    if row['person1'] not in id_score_s: 
        id_score_s[row['person1']] = row['score1']
    if row['person2'] not in id_score_s: 
        id_score_s[row['person2']] = row['score2']  

In [422]:
len(id_score_s.keys())

25854

In [35]:
### pickle id_score_s
with open('id_score_s.pkl', 'wb') as file:
    pickle.dump(id_score_s, file)

#### build network with small dataset ####

In [16]:
net_s = nx.Graph()

In [17]:
for index, row in foes_s.iterrows():
    net_s.add_edge(row['person1'], row['person2'], weight= -1)

In [24]:
net_s.number_of_nodes()    # same as # of keys in the dict

25854

In [25]:
net_s.number_of_edges()

142589

In [21]:
list(net_s.nodes)[:5]

['3e9972af_722e8b21',
 'f9fce742_9e6fd13d',
 '18c5cff6_ea4f0c22',
 '7c57d81d_9e0e3e7a',
 '86e62183_ad41fc40']

In [20]:
net_s.edges['8b28e2cf_2b895a86', '1fca0d07_adc649b8']['weight']

-1

In [23]:
for index, row in friends_s.iterrows():
    net_s.add_edge(row['person1'], row['person2'], weight=1)

In [36]:
# add score to vertices 
for key, score in id_score_s.items():
    net_s.nodes[key]['score'] = score

In [38]:
net_s.nodes['8b28e2cf_2b895a86']   # verified

{'score': -1.0}

In [39]:
# pickle net_s
with open('net_s.pkl', 'wb') as file:
    pickle.dump(net_s, file)

In [40]:
# find triangles in net
cycls_3 = [c for c in nx.cycle_basis(net_s) if len(c)==3]

In [43]:
len(cycls_3)

34751

In [46]:
#c0 = cycls_3[0]

In [52]:
#pairs = list(itertools.combinations(c0, 2))

In [53]:
#pairs[0]

('bdac1eff_9a4ecb4c', 'ba0f6006_5551ee80')

In [75]:
# mul = net_s.edges[pairs[0][0], pairs[0][1]]['weight']
# for pair in pairs[1:]: 
#     mul *= net_s.edges[pair[0], pair[1]]['weight']

In [76]:
# mul

1

In [65]:
net_s.nodes['8b28e2cf_2b895a86']['score']

-1.0

In [45]:
net_s.edges['8b28e2cf_2b895a86', '1fca0d07_adc649b8']['weight']

-1

In [162]:
# a = 1
# x1 = -1.0
# x2 = 0.0
# x3 = 1.0
# if a == 1 and (x1 == x2 == x3) == True:
#     print 'yes'
# else:
#     print 'no'

In [169]:
# 1st round: (friend, friend, foe) extracted
wrong_p1 = []

In [170]:
for cycle in cycls_3:
    pairs = list(itertools.combinations(cycle, 2))
    # multiplicaiton of 3 weights: initialize mul with first weight
    mul = net_s.edges[pairs[0][0], pairs[0][1]]['weight']
    for pair in pairs[1:]: 
        mul *= net_s.edges[pair[0], pair[1]]['weight']
    
    if (mul == -1) and ((net_s.edges[pairs[0][0], pairs[0][1]]['weight'] + \
                      net_s.edges[pairs[1][0], pairs[1][1]]['weight'] + \
                      net_s.edges[pairs[2][0], pairs[2][1]]['weight']) == 1):
        wrong_p1.append(cycle)

In [171]:
len(wrong_p1)

538

In [177]:
# p2 = wrong_p1[1]

In [178]:
# p2

['b3d276a7_f7faaee1', '9958c5f5_d6819a41', 'c4098202_59ca60c0']

In [179]:
# # verify wrong_p1[0] by condition
# pairs = list(itertools.combinations(p2, 2))
# mul = net_s.edges[pairs[0][0], pairs[0][1]]['weight']
# for pair in pairs[1:]: 
#     mul *= net_s.edges[pair[0], pair[1]]['weight']
# mul

-1

In [186]:
# print net_s.edges[pairs[0][0], pairs[0][1]]['weight']
# print net_s.edges[pairs[1][0], pairs[1][1]]['weight']
# print net_s.edges[pairs[2][0], pairs[2][1]]['weight']

-1
1
1


In [181]:
with open('wrong_p1.pkl', 'wb') as file:
    pickle.dump(wrong_p1, file)

In [187]:
with open('wrong_p1.pkl', 'rb') as file:
    p1 = pickle.load(file)

In [189]:
p1[0]

['8fa85ae_47e6744b', '9958c5f5_d6819a41', 'c4098202_59ca60c0']

In [190]:
# further(2nd) narrow down: all vertices in a cycle must have known scores.
wrong_p2 = []

In [191]:
for cycle in p1:
    if net_s.nodes[cycle[0]]['score'] * net_s.nodes[cycle[1]]['score'] * net_s.nodes[cycle[2]]['score'] != 0:
        wrong_p2.append(cycle)

In [192]:
len(wrong_p2)

52

In [193]:
wrong_p2[0]

['c4a4c0e6_c3ac6729', '21fc5e24_72210188', '81007e87_f944e95b']

In [203]:
# further(3rd) narrow down: by rule 1, keep cycles with bad + good person friends
wrong_p3 = []

In [204]:
for cycle in wrong_p2:
    pairs = list(itertools.combinations(cycle, 2))
    # product of 3 weights: initialize mul with first weight
    for pair in pairs:
        if (net_s.edges[pair[0], pair[1]]['weight'] == 1) and (net_s.nodes[pair[0]]['score'] * net_s.nodes[pair[1]]['score'] < 0):                      
            wrong_p3.append(cycle)
            break

In [205]:
len(wrong_p3)

42

In [212]:
wrong_p3_l = []
for item in wrong_p3:
    wrong_p3_l.extend(item)

In [253]:
# get 5 most frequent vertices in the cycyles
sorted(Counter(wrong_p3_l).items(), key = lambda (k,v): (v,k), reverse = True)[:5]

[('c4098202_59ca60c0', 15),
 ('c4a4c0e6_c3ac6729', 8),
 ('81007e87_f944e95b', 6),
 ('556ba434_c0ca2371', 6),
 ('433224d2_8de5da2e', 4)]

In [263]:
wrong_p4 = []

In [264]:
for item in wrong_p3:
    if 'c4098202_59ca60c0' in item:
        wrong_p4.append(item)

In [265]:
wrong_p4

[['9779191_d458ccd1', '6be100d7_532794ea', 'c4098202_59ca60c0'],
 ['54cf58aa_f951603c', '790f0441_162b707a', 'c4098202_59ca60c0'],
 ['ba850aa4_635b1b38', '4b1efc02_d975992f', 'c4098202_59ca60c0'],
 ['eb15b11c_2086cdf0', '7295669_c50f4599', 'c4098202_59ca60c0'],
 ['d34748d2_55b76015', '19fb72ee_3184949e', 'c4098202_59ca60c0'],
 ['d34748d2_55b76015', '8fd3a344_70daee4f', 'c4098202_59ca60c0'],
 ['4ef00044_557c7c45', 'd430f708_c07ee38b', 'c4098202_59ca60c0'],
 ['9779191_d458ccd1', 'a2f656c3_2e9d2b6d', 'c4098202_59ca60c0'],
 ['14fc472a_f4b57e09', '350b2065_99e3747c', 'c4098202_59ca60c0'],
 ['2af5a458_d1345878', '54cf58aa_f951603c', 'c4098202_59ca60c0'],
 ['4104c504_b2a0fab4', '256649c4_29402af', 'c4098202_59ca60c0'],
 ['2af5a458_d1345878', '8fa85ae_47e6744b', 'c4098202_59ca60c0'],
 ['258e3ae3_afaf2804', 'befd2359_2a206c18', 'c4098202_59ca60c0'],
 ['258e3ae3_afaf2804', '52907b87_aa3e7910', 'c4098202_59ca60c0'],
 ['258e3ae3_afaf2804', 'd34748d2_55b76015', 'c4098202_59ca60c0']]

In [267]:
w_s = []
for cycle in wrong_p4:
    w1 = net_s.edges[cycle[0], 'c4098202_59ca60c0']['weight']
    w2 = net_s.edges[cycle[1], 'c4098202_59ca60c0']['weight']
    w_s.append((w1, w2))

In [268]:
w_s

[(1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1)]

In [269]:
score_s = []
for cycle in wrong_p4:
    s1 = net_s.nodes[cycle[0]]['score']
    s2 = net_s.nodes[cycle[1]]['score']
    score_s.append((s1, s2))

In [270]:
score_s

[(2.0, -1.0),
 (1.0, -2.0),
 (-2.0, 1.0),
 (2.0, -2.0),
 (-2.0, 2.0),
 (-2.0, 1.0),
 (2.0, -2.0),
 (2.0, -2.0),
 (3.0, -2.0),
 (-2.0, 1.0),
 (-2.0, 2.0),
 (-2.0, 2.0),
 (2.0, -2.0),
 (2.0, -3.0),
 (2.0, -2.0)]

## Part (b)

In [357]:
# create a dictionary with id_score in score table: this table with be expanded later. 
id_score = {}
for index, row in scores.iterrows():
    id_score[row['person id']] = row['score']

In [363]:
with open('id_score_full.pkl', 'wb') as file:
    pickle.dump(id_score, file)

In [395]:
with open('id_score_full.pkl', 'rb') as file:
    id_score = pickle.load(file)

In [396]:
len(id_score.items())

1927

### testing algorithm on small dataset generated in part (a)

In [390]:
with open('net_s.pkl', 'rb') as file:
    net_s = pickle.load(file)

In [391]:
# net_s: update weights of 8 edges identified in part (a)
net_s.edges['8fa85ae_47e6744b', 'c4098202_59ca60c0']['weight'] = 0
net_s.edges['8fd3a344_70daee4f', 'c4098202_59ca60c0']['weight'] = 0
net_s.edges['9779191_d458ccd1', 'c4098202_59ca60c0']['weight'] = 0
net_s.edges['4b1efc02_d975992f', 'c4098202_59ca60c0']['weight'] = 0
net_s.edges['eb15b11c_2086cdf0', 'c4098202_59ca60c0']['weight'] = 0
net_s.edges['256649c4_29402af', 'c4098202_59ca60c0']['weight'] = 0
net_s.edges['14fc472a_f4b57e09', 'c4098202_59ca60c0']['weight'] = 0
net_s.edges['4ef00044_557c7c45', 'c4098202_59ca60c0']['weight'] = 0

In [392]:
net_s.number_of_edges()      # verified: same as before

142589

In [393]:
net_s.number_of_nodes()

25854

In [410]:
len(id_score.keys())

1987

In [398]:
# Add an new attribute to every node in net_s
for node in list(net_s.nodes):
    if node in id_score:
        net_s.nodes[node]['known_neighbor'] = 0   # already in the score table
    else:
        k_n = 0
        for adj_item in list(net_s.adj[node]):
           if adj_item in id_score:
               k_n += 1
        net_s.nodes[node]['known_neighbor'] = k_n    # k_n >= 1

In [364]:
#len(wrong_p1)

In [399]:
# decrease friend weights in triangles in wrong_p1: by rule(2) and rule(3)
for cycle in wrong_p1:
    pairs = list(itertools.combinations(cycle, 2))
    for pair in pairs:
        if net_s.edges[pair[0], pair[1]]['weight'] == 1:
            net_s.edges[pair[0], pair[1]]['weight'] = 0.001

In [301]:
#net_s.node['c4098202_59ca60c0']

{'known_neighbor': 0, 'score': -2.0}

In [400]:
# sort vertices by their know_neighbor
node_heap = []
for node in list(net_s.nodes):
    if net_s.node[node]['known_neighbor'] != 0:    # never put nodes with known score in the heapq
        heapq.heappush(node_heap, (net_s.node[node]['known_neighbor'], node))

In [401]:
len(node_heap)   # 25854 - 23930 = 1924 < 1927 

23930

In [402]:
heapq.nlargest(1, node_heap)

[(154, '48b40daa_41326021')]

In [405]:
net_s.degree('48b40daa_41326021')

154

In [309]:
# node_heap.remove((154, '48b40daa_41326021'))
# heapq.heapify(node_heap)

In [408]:
# calcualte a node score, update node attribute and the score dictionary, create a new heapq
begin = time.time()
print begin

while node_heap:
    largest = heapq.nlargest(1, node_heap)
    current_node = largest[0][1]
    new_score = 0
    count = 0
    for adj_item in list(net_s.adj[current_node]):
        if net_s.node[adj_item]['known_neighbor'] == 0:    # node's score is known or has been updated
            new_score += net_s.edges[adj_item, current_node]['weight'] * net_s.node[adj_item]['score']
            count += 1
        else:
            net_s.node[adj_item]['known_neighbor'] += 1

    new_score /= count
    #ratio = 1.0 * count / net_s.degree[current_node]
    #print ratio
    
    # set 'known_neighbor' to 0
    net_s.nodes[current_node]['known_neighbor'] = 0

    # update node score in net, expand id_score dictionary
    net_s.nodes[current_node]['score'] = new_score
    id_score[current_node] = new_score

    # create a new node_heap:
    node_heap = []
    for node in list(net_s.nodes):
        if net_s.node[node]['known_neighbor'] != 0:    # never put nodes with known score in the heapq
            heapq.heappush(node_heap, (net_s.node[node]['known_neighbor'], node))
    
end = time.time()
print end
print end - begin
#     # remove it from heapq and reheapify
#     node_heap.remove(largest)
#     heapq.heapify(node_heap)    

In [382]:
id_score['c4098202_59ca60c0']   # verified: no change for node in the score table

-2

In [383]:
id_score.items()[:5]

[('c154494_12225688', -2.2),
 ('9f5eff3b_36cc13a2', 2.0),
 ('601d4aa3_628958d5', -1.4444444444444444),
 ('8f6bc512_664f5624', -1.5),
 ('7906d541_f32b96ea', -2)]

In [ ]:
# expand net_s with rest of rows in friends and foes

# identify (f, f, t) triangles in net_full

In [319]:
node_heap

AttributeError: 'list' object has no attribute 'keys'

In [330]:
#heapq.heappush(h, (8, 'release product'))

In [331]:
heapq.nlargest(1, h)

[(7, 'release product')]

In [407]:
h

[(3, 'create tests'),
 (7, 'release product'),
 (5, 'write code'),
 (8, 'release product')]

In [333]:
heapq.heapreplace(h, (8, 'release product'))

(1, 'write spec')

### apply algorithm on full dataset (friend + foe)

*Note: deleting 259 (good, good) pairs in foe table will not decrease # of nodes in the full network, because their scores are known*


In [411]:
net = nx.Graph()

In [412]:
with open('foes_true.pkl', 'rb') as file:
    foes = pickle.load(file)

In [413]:
with open('friends_new.pkl', 'rb') as file:
    friends = pickle.load(file)

In [415]:
len(friends)

474835

In [472]:
with open('id_score_full.pkl', 'rb') as file:
    id_score = pickle.load(file)

In [473]:
len(id_score)

1927

In [418]:
for index, row in foes.iterrows():
    net.add_edge(row['person1'], row['person2'], weight= -1)

In [419]:
for index, row in friends.iterrows():
    net.add_edge(row['person1'], row['person2'], weight = 1)

In [430]:
net.number_of_nodes()    # same as # of keys in the dict

54270

In [431]:
net.number_of_edges()

609924

In [432]:
# add score and known_neighbor to vertices
for node in list(net.nodes):
    if node in id_score:
        net.nodes[node]['known_neighbor'] = -1   # already in the score table
        net.nodes[node]['score'] = id_score[node]
    elif node in id_score_s:     # id_score_s is the dict with small dateset 
        k_n = 0
        for adj_item in list(net.adj[node]):
           if adj_item in id_score:
               k_n += 1
        net.nodes[node]['known_neighbor'] = k_n
        net.nodes[node]['score'] = 0.0
    else:      # not in small dataset
        net.nodes[node]['known_neighbor'] = 0
        net.nodes[node]['score'] = 0.0

In [433]:
net.nodes['b0177400_be9028c7'] 

{'known_neighbor': -1, 'score': -2}

In [435]:
# update weight: 8 false friendships
net.edges['8fa85ae_47e6744b', 'c4098202_59ca60c0']['weight'] = 0
net.edges['8fd3a344_70daee4f', 'c4098202_59ca60c0']['weight'] = 0
net.edges['9779191_d458ccd1', 'c4098202_59ca60c0']['weight'] = 0
net.edges['4b1efc02_d975992f', 'c4098202_59ca60c0']['weight'] = 0
net.edges['eb15b11c_2086cdf0', 'c4098202_59ca60c0']['weight'] = 0
net.edges['256649c4_29402af', 'c4098202_59ca60c0']['weight'] = 0
net.edges['14fc472a_f4b57e09', 'c4098202_59ca60c0']['weight'] = 0
net.edges['4ef00044_557c7c45', 'c4098202_59ca60c0']['weight'] = 0

In [434]:
# identify (friend, friend, foe)
cycls_3 = [c for c in nx.cycle_basis(net) if len(c)==3]
tri = []
for cycle in cycls_3:
    pairs = list(itertools.combinations(cycle, 2))
    # multiplicaiton of 3 weights: initialize mul with first weight
    mul = net.edges[pairs[0][0], pairs[0][1]]['weight']
    for pair in pairs[1:]: 
        mul *= net.edges[pair[0], pair[1]]['weight']
    
    if (mul == -1) and ((net.edges[pairs[0][0], pairs[0][1]]['weight'] + \
                      net.edges[pairs[1][0], pairs[1][1]]['weight'] + \
                      net.edges[pairs[2][0], pairs[2][1]]['weight']) == 1):
        tri.append(cycle)

In [436]:
len(tri)

5145

In [437]:
# update weights: decrease friend weights in triangles in wrong_p1: by rule(2) and rule(3)
for cycle in tri:
    pairs = list(itertools.combinations(cycle, 2))
    for pair in pairs:
        if net.edges[pair[0], pair[1]]['weight'] == 1:
            net.edges[pair[0], pair[1]]['weight'] = 0.001

In [438]:
# finish initilization: pickle net
with open('net_full.pkl', 'wb') as file:
    pickle.dump(net, file)

In [471]:
with open('net_full.pkl', 'rb') as file:
    net = pickle.load(file)

In [474]:
# sort vertices by their know_neighbor
node_heap = []
for node in list(net.nodes):
    if net.node[node]['known_neighbor'] > 0:    # neither with known score nor 0 neighbors
        heapq.heappush(node_heap, (1.0 * net.node[node]['known_neighbor'] / net.degree(node) , node))

In [467]:
len(node_heap)   # same as in net_s: make sense :)

23930

In [475]:
heapq.nlargest(1, node_heap)  # verified

[(1.0, 'ffcfa365_2925a34a')]

In [470]:
net.degree('ffcfa365_2925a34a')

1

In [476]:
# calcualte a node score, update node attribute and the score dictionary, create a new heapq
begin = time.time()

while node_heap:
    largest = heapq.nlargest(1, node_heap)
    current_node = largest[0][1]
    new_score = 0
    count = 0
    for adj_item in list(net.adj[current_node]):
        if net.node[adj_item]['known_neighbor'] == -1:    # node's score is known or has been updated
            new_score += net.edges[adj_item, current_node]['weight'] * net.node[adj_item]['score']
            count += 1
        else:
            net.node[adj_item]['known_neighbor'] += 1     # update adjacent nodes 'known_neighbor' 
            
    new_score /= count
    # set 'known_neighbor' to -1
    net.nodes[current_node]['known_neighbor'] = -1

    # update node score in net, expand id_score dictionary
    net.nodes[current_node]['score'] = new_score
    id_score[current_node] = new_score

    # create a new node_heap:
    node_heap = []
    for node in list(net.nodes):
        if net.node[node]['known_neighbor']  > 0:    # never put nodes with known score in the heapq
            heapq.heappush(node_heap, (1.0 * net.node[node]['known_neighbor'] / net.degree(node), node))

            
end = time.time()
print end - begin
#     # remove it from heapq and reheapify
#     node_heap.remove(largest)
#     heapq.heapify(node_heap)    

8399.92395902


In [477]:
len(id_score)

54238

In [478]:
with open('id_score_final.pkl', 'wb') as file:
    pickle.dump(id_score, file)

In [481]:
id_score['1122b7d5_ed317db7']

-2

### Fix bug: difference between node_calculated and node_in_net

In [482]:
node_calculated = set(id_score.keys())

In [485]:
node_in_net = set(net.nodes)

In [486]:
len(node_calculated)

54238

In [487]:
len(node_in_net)

54270

In [488]:
diff = node_in_net - node_calculated

In [509]:
len(diff)

35

In [489]:
diff

{'14ddcddd_b241f4fe',
 '1a432144_20b65eac',
 '2233fffe_54f7c149',
 '25f2e9e8_b77ac4f9',
 '2b853001_b808a505',
 '34c6a3cf_6e4e7899',
 '4056adfb_f0f26907',
 '4933062c_c5392638',
 '4f5344cd_1deb2d6a',
 '4fe8941c_be0c0106',
 '54ac52db_ba531816',
 '575fd620_c323b529',
 '5c6563b_da9ff8fd',
 '62ef4352_b8d29643',
 '630bac79_f5352bb4',
 '63dfbea0_55192781',
 '64c0ec72_bc003d2d',
 '68b4106f_ad0f9ce9',
 '73222626_730b5252',
 '7998bfab_a7a3307f',
 '9206ff2f_8fb7389b',
 '98c0ab41_d77bbc8e',
 '9a4d976_1dbaab6',
 'b09a1db1_99979657',
 'b355ab62_3a5540d9',
 'ccac477c_6444f693',
 'ccd9ca6_2e739fff',
 'cdbbfbff_e5d760ad',
 'd3fa21db_56173d26',
 'd656575f_21b9431b',
 'd987246f_cd15cb0',
 'f1d6ca15_e8592bc5',
 'f3511315_1399fee1',
 'f72e44b3_920c913d',
 'feb31af3_b4dac040'}

In [490]:
net.nodes['14ddcddd_b241f4fe']

{'known_neighbor': 0, 'score': 0.0}

In [493]:
net.nodes['73222626_730b5252']

{'known_neighbor': 0, 'score': 0.0}

In [499]:
# create dataframe from dictionary 
classification = pd.DataFrame(id_score.items(), columns=['ID', 'score'])

In [500]:
len(classification)

54238

In [501]:
cls = pd.DataFrame([[ID, 0.0] for ID in diff], columns=['ID', 'score'])

In [505]:
classification = classification.append(cls)

In [506]:
len(classification)   # make sense

54273

In [508]:
len(set(classification['ID']))  # no duplicate

54273

In [513]:
#classification.index.tolist()[-5:]

In [514]:
classification.reset_index(drop=True, inplace = True)

In [515]:
classification['cls'] = ""

In [516]:
classification['cls'][classification['score'] < 0] = 'bad'

/Users/huilinhu/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [517]:
classification['cls'][classification['score'] > 0] = 'good'

/Users/huilinhu/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [518]:
classification['cls'].value_counts()

bad     30439
good    23707
          127
Name: cls, dtype: int64

In [519]:
classification['cls'][classification['score'] == 0] = 'bad'

/Users/huilinhu/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [520]:
classification['cls'].value_counts()

bad     30566
good    23707
Name: cls, dtype: int64

In [521]:
with open('classification.pkl', 'wb') as file:
    pickle.dump(classification, file)

In [522]:
classification.to_csv('cls.csv')